Model

In [1]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [2]:
#checking for device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
print(device)

cpu


In [4]:
transformer = transforms.Compose([ 
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),  
    transforms.ToTensor(), 
    transforms.Normalize([0.5,0.5,0.5],
                        [0.5,0.5,0.5]) 
])

In [6]:
#path for training and testing
train_path = 'D://program//pytorch//sportsball/dataset/train'
test_path = 'D://program//pytorch//sportsball/dataset/test'

In [7]:
#dataloade
train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size = 64, shuffle=True
)
test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size = 32, shuffle=True
)

In [8]:
#category
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['american_football', 'baseball', 'basketball', 'billiard_ball', 'bowling_ball', 'cricket_ball', 'football', 'golf_ball', 'hockey_ball', 'hockey_puck', 'rugby_ball', 'shuttlecock', 'table_tennis_ball', 'tennis_ball', 'volleyball']


In [18]:
print(len(classes))

15


In [33]:
#cnn network
class ConvNet(nn.Module):
    def __init__(self, num_classes = 15):
        super(ConvNet,self).__init__()
        self.conv1 = nn.Conv2d(in_channels= 3,out_channels =12,kernel_size=3,stride=1,padding=1) 
        self.bn1 = nn.BatchNorm2d(num_features=12)
        self.relu1 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2)
        
        self.conv2 = nn.Conv2d(in_channels= 12,out_channels =20,kernel_size=3,stride=1,padding=1) 
        self.relu2 = nn.ReLU() 

        self.conv3 = nn.Conv2d(in_channels= 20,out_channels =32,kernel_size=3,stride=1,padding=1) 
        self.bn3 = nn.BatchNorm2d(num_features=32)
        self.relu3 = nn.ReLU()  

        self.conv4 = nn.Conv2d(in_channels= 32,out_channels =64,kernel_size=3,stride=1,padding=1) 
        self.bn4 = nn.BatchNorm2d(num_features=64)
        self.relu4 = nn.ReLU()  


        self.fc =nn.Linear(in_features=75 * 75* 64 ,out_features=num_classes)

        #feed forward function

    def forward(self,input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        output = self.pool(output)
        output = self.conv2(output)
        output = self.relu2(output)
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)

        output = self.conv4(output)
        output = self.bn4(output)
        output = self.relu4(output)

            #Above output will be in matrix  form (256,32,75,75)

        output= output.view(-1,64*75*75)
        output = self.fc(output)

        return output

In [34]:
model =ConvNet(num_classes=15).to(device)

In [35]:
#Optimizer and loss function

optimizer = Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function =nn.CrossEntropyLoss()

In [36]:
num_epochs = 20

In [37]:
#calculate the size of training and testing images

train_count = len(glob.glob(train_path+'/**/*.jpg'))
test_count = len(glob.glob(test_path+'/**/*.jpg'))

In [38]:
print(train_count,test_count)

7328 1841


In [39]:
#model training and saving best model
best_accuracy = 0.0
for epoch in range(num_epochs):

    #evaluation and training on training dataset
    model.train()
    train_accuracy =0.0
    train_loss = 0.0

    for i , (images,labels) in enumerate (train_loader):
        if torch.cuda.is_available():
            images= Variable(images.cuda())
            labels=Variable(labels.cuda())
        optimizer.zero_grad()  #backprop accumulates gradients, and not want them in minibatches
        outputs = model(images)

        loss = loss_function(outputs,labels)
        loss.backward()
        optimizer.step() #update weight and bias

        train_loss += loss.cpu().data*images.size(0)    
        _,prediction = torch.max(outputs.data,1)   
        train_accuracy +=int(torch.sum(prediction == labels.data))
    train_accuracy = train_accuracy/train_count
    train_loss = train_loss/train_count

    #evalutaion on testing dataset

    model.eval()

    test_accuracy =0.0
    for i , (images,labels) in enumerate (test_loader):
        if torch.cuda.is_available():
            images= Variable(images.cuda())
            labels=Variable(labels.cuda())
        
        outputs=model(images)
        _,prediction = torch.max(outputs.data,1)   
        test_accuracy +=int(torch.sum(prediction == labels.data))
    test_accuracy = test_accuracy/test_count

    print('Epoch :' +str(epoch)+' Train loss:'+str(int(train_loss))+' Train accuracy:'+str(train_accuracy)+' Test accuracy:'+str(test_accuracy))

    #save the best model

    if test_accuracy >best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy


Epoch :0Train loss:31Train accuracy:0.13960152838427947Test accuracy:0.14774579033134166
Epoch :1Train loss:5Train accuracy:0.17098799126637554Test accuracy:0.12167300380228137
Epoch :2Train loss:3Train accuracy:0.1019377729257642Test accuracy:0.08962520369364475
Epoch :3Train loss:2Train accuracy:0.10575873362445415Test accuracy:0.06844106463878327
Epoch :4Train loss:2Train accuracy:0.12254366812227074Test accuracy:0.09397066811515481
Epoch :5Train loss:2Train accuracy:0.11340065502183407Test accuracy:0.07224334600760456
Epoch :6Train loss:2Train accuracy:0.1204967248908297Test accuracy:0.06626833242802825
Epoch :7Train loss:2Train accuracy:0.14519650655021835Test accuracy:0.11461162411732755
Epoch :8Train loss:2Train accuracy:0.18681768558951964Test accuracy:0.11841390548614883
Epoch :9Train loss:2Train accuracy:0.26569323144104806Test accuracy:0.07332971211298207
Epoch :10Train loss:2Train accuracy:0.36067139737991266Test accuracy:0.12927756653992395
Epoch :11Train loss:1Train accur

Inference - testing on datas

In [125]:
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
from PIL import Image
import cv2

In [126]:
train_path = 'D://program//pytorch//sportsball//dataset//train'
pred_path = 'D://program//pytorch//sportsball//dataset//pred'

In [127]:
#category
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['american_football', 'baseball', 'basketball', 'billiard_ball', 'bowling_ball', 'cricket_ball', 'football', 'golf_ball', 'hockey_ball', 'hockey_puck', 'rugby_ball', 'shuttlecock', 'table_tennis_ball', 'tennis_ball', 'volleyball']


In [128]:
class ConvNet(nn.Module):
    def __init__(self, num_classes = 15):
        super(ConvNet,self).__init__()
        self.conv1 = nn.Conv2d(in_channels= 3,out_channels =12,kernel_size=3,stride=1,padding=1) 
        self.bn1 = nn.BatchNorm2d(num_features=12)
        self.relu1 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2)
        
        self.conv2 = nn.Conv2d(in_channels= 12,out_channels =20,kernel_size=3,stride=1,padding=1) 
        self.relu2 = nn.ReLU() 

        self.conv3 = nn.Conv2d(in_channels= 20,out_channels =32,kernel_size=3,stride=1,padding=1) 
        self.bn3 = nn.BatchNorm2d(num_features=32)
        self.relu3 = nn.ReLU()  

        self.conv4 = nn.Conv2d(in_channels= 32,out_channels =64,kernel_size=3,stride=1,padding=1) 
        self.bn4 = nn.BatchNorm2d(num_features=64)
        self.relu4 = nn.ReLU()  


        self.fc =nn.Linear(in_features=75 * 75* 64 ,out_features=num_classes)

        #feed forward function

    def forward(self,input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        output = self.pool(output)
        output = self.conv2(output)
        output = self.relu2(output)
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)

        output = self.conv4(output)
        output = self.bn4(output)
        output = self.relu4(output)

            #Above output will be in matrix  form (256,32,75,75)

        output= output.view(-1,64*75*75)
        output = self.fc(output)

        return output

In [129]:
checkpoint =torch.load('best_checkpoint.model')
model = ConvNet(num_classes=15)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu4): ReLU()
  (fc): Linear(in_features=360000, out_features=15, bias=True)
)

In [130]:
transformer = transforms.Compose([ 
    transforms.Resize((150,150)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],
                      [0.5,0.5,0.5])
])

In [131]:
#prediction fuction
def prediction(img_path,transformer):
    image = Image.open(img_path)
    image_tensor = transformer(image).float()
    image_tensor =image_tensor.unsqueeze_(0)

    if torch.cuda.is_available():
        image_tensor.cuda()
    input = Variable(image_tensor)

    output = model(input)
    index=output.data.numpy().argmax()
    pred=classes[index]
    return pred



In [132]:
images_path = glob.glob(pred_path+'/*.jpg')

In [ ]:
pred_dict={} #image name as key , value as prediction
for i in images_path:
    pred_dict[i[i.rfind('/')+1:]] = prediction(i,transformer)

In [134]:
pred_dict

{'pred\\0 (1).jpg': 'hockey_puck',
 'pred\\0 (10) - Copy.jpg': 'hockey_puck',
 'pred\\0 (10).jpg': 'hockey_puck',
 'pred\\0 (100).jpg': 'football',
 'pred\\0 (101).jpg': 'football',
 'pred\\0 (102).jpg': 'football',
 'pred\\0 (103).jpg': 'football',
 'pred\\0 (104).jpg': 'football',
 'pred\\0 (105).jpg': 'football',
 'pred\\0 (106).jpg': 'football',
 'pred\\0 (107).jpg': 'football',
 'pred\\0 (108).jpg': 'football',
 'pred\\0 (109).jpg': 'volleyball',
 'pred\\0 (11) - Copy.jpg': 'rugby_ball',
 'pred\\0 (11).jpg': 'rugby_ball',
 'pred\\0 (110).jpg': 'golf_ball',
 'pred\\0 (111).jpg': 'golf_ball',
 'pred\\0 (112).jpg': 'golf_ball',
 'pred\\0 (113).jpg': 'golf_ball',
 'pred\\0 (114).jpg': 'golf_ball',
 'pred\\0 (115).jpg': 'golf_ball',
 'pred\\0 (116).jpg': 'golf_ball',
 'pred\\0 (117).jpg': 'golf_ball',
 'pred\\0 (118).jpg': 'golf_ball',
 'pred\\0 (119).jpg': 'golf_ball',
 'pred\\0 (12).jpg': 'rugby_ball',
 'pred\\0 (120).jpg': 'golf_ball',
 'pred\\0 (121).jpg': 'tennis_ball',
 'pred\\0 